In [132]:
import numpy as np

In [133]:
from pandas import Series, DataFrame

In [134]:
import pandas as pd

In [135]:
from pandas import read_html

In [136]:
url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
url_wb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'

In [137]:
dframe_qb = pd.io.html.read_html(url_qb_season17)
dframe_rb = pd.io.html.read_html(url_rb_season17)
dframe_wb = pd.io.html.read_html(url_wb_season17)
dframe_te = pd.io.html.read_html(url_te_season17)
dframe_def = pd.io.html.read_html(url_def_season17)

In [395]:
qb = dframe_qb[0]
rb = dframe_rb[0]
rb.describe()
rb.iloc[0]

                     Player    Todd GurleyT. Gurley
                     Bye                          8
                     Pts*                       304
Passing              Att                          0
Rushing              Cmp                          0
Receiving            Yds                          0
Fumbles              TD                           0
Unnamed: 7_level_0   Int                          0
Unnamed: 8_level_0   2Pt                          0
Unnamed: 9_level_0   Att                        279
Unnamed: 10_level_0  Yds                       1305
Unnamed: 11_level_0  TD                          13
Unnamed: 12_level_0  2Pt                          0
Unnamed: 13_level_0  Rec                         64
Unnamed: 14_level_0  Yds                        788
Unnamed: 15_level_0  TD                           6
Unnamed: 16_level_0  2Pt                          0
Unnamed: 17_level_0  FL                           2
Unnamed: 18_level_0  TD                           0
Name: 0, dty

In [396]:
#Matrx to hold data 
dataMatrix = []

In [397]:
#Save Data from 
bye = []
data = qb['\xa0'].get("Bye")
for i in data:
    bye.append(i)
    
dataMatrix.append(bye)


pts = []
data = qb['\xa0'].get("Pts*")
for i in data:
    pts.append(i)
    
dataMatrix.append(pts)

#Get Passing attempt from Att Data 
passingAtt = []
data = qb['Passing'].get("Att")
for i in data:
    passingAtt.append(i)
    
dataMatrix.append(passingAtt)


passingCmp = []
data = qb['Rushing'].get('Cmp')
for i in data:
    passingCmp.append(i)
    
dataMatrix.append(passingCmp)


passingYds = []
data = qb['Receiving'].get('Yds')
for i in data:
    passingYds.append(i)
    
dataMatrix.append(passingYds)

passingTD = []
data = qb['Fumbles'].get('TD')
for i in data:
    passingTD.append(i)
    
dataMatrix.append(passingTD)

passingInt = []
data = qb['Unnamed: 7_level_0'].get('Int')
for i in data:
    passingInt.append(i)

dataMatrix.append(passingInt)

passing2Pt = []
data = qb['Unnamed: 8_level_0'].get('2Pt')
for i in data:
    passing2Pt.append(i)
    
dataMatrix.append(passing2Pt)

rushingAtt = []
data = qb['Unnamed: 9_level_0'].get('Att')
for i in data:
    rushingAtt.append(i)
    
dataMatrix.append(rushingAtt)

rushingYds = []
data = qb['Unnamed: 10_level_0'].get('Yds')
for i in data:
    rushingYds.append(i)
    
dataMatrix.append(rushingYds)

rushingTD = []
data = qb['Unnamed: 11_level_0'].get('TD')
for i in data:
    rushingTD.append(i)
    
dataMatrix.append(rushingTD)

rushing2Pt = []
data = qb['Unnamed: 12_level_0'].get('2Pt')
for i in data:
    rushing2Pt.append(i)
    
dataMatrix.append(rushing2Pt)

receivingRec = []
data = qb['Unnamed: 13_level_0'].get('Rec')
for i in data:
    receivingRec.append(i)
    
dataMatrix.append(receivingRec)

receivingYds = []
data = qb['Unnamed: 14_level_0'].get('Yds')
for i in data:
    receivingYds.append(i)
    
dataMatrix.append(receivingYds)

receivingTD = []
data = qb['Unnamed: 15_level_0'].get('TD')
for i in data:
    receivingTD.append(i)
    
dataMatrix.append(receivingTD)

receiving2Pt = []
data = qb['Unnamed: 16_level_0'].get('2Pt')
for i in data:
    receiving2Pt.append(i)
    
dataMatrix.append(receiving2Pt)

fumblesFL = []
data = qb['Unnamed: 17_level_0'].get('FL')
for i in data:
    fumblesFL.append(i)
    
dataMatrix.append(fumblesFL)

fumblesTD = []
data = qb['Unnamed: 18_level_0'].get('TD')
for i in data:
    fumblesTD.append(i)
    
dataMatrix.append(fumblesTD)

In [398]:
#Get name from Player column and save to nameIndex
NAME_FIX_OFFSET = 3

nameIndex = []
data = qb['\xa0'].get("Player")
for i in data:
    nameIndex.append(i)
    
    
size = len(nameIndex)

for i in range(size):
    a1 = nameIndex[i]
    count = 0
    
    for x in a1:
        count += 1
        if x == u'\xa0':
            break
            
    count -= NAME_FIX_OFFSET
    nameIndex[i] = a1[:count]




In [399]:
dataMatrix = np.array(dataMatrix)
dataMatrix = dataMatrix.transpose()

In [400]:

#create a tuple for multi-indexed Dataframe
tup = [('', 'Bye'),
       ('', 'Pts'),
       ('Passing', 'Att'),
       ('Passing', 'Cmp'),
       ('Passing', 'Yds'),
       ('Passing', 'TD'),
       ('Passing', 'Int'),
       ('Passing', '2Pt'),
       ('Rushing', 'Att'),
       ('Rushing', 'Yds'),
       ('Rushing', 'TD'),
       ('Rushing', '2Pt'),
       ('Receiving', 'Rec'),
       ('Receiving', 'Yds'),
       ('Receiving', 'TD'),
       ('Receiving', '2Pt'),
       ('Fumbles', 'FL'),
       ('Fumbles', 'TD')
      ]



In [401]:
index = pd.MultiIndex.from_tuples(tup)


In [402]:
dframe1 = DataFrame(dataMatrix, columns = index, index = nameIndex)


In [403]:
dframe1

Passing                                  \
                     Bye    Pts     Att    Cmp     Yds    TD   Int  2Pt   
Russell Wilson       6.0  402.0   553.0  339.0  3983.0  34.0  11.0  2.0   
Tom Brady            9.0  352.0   581.0  385.0  4577.0  32.0   8.0  2.0   
Carson Wentz        10.0  338.0   440.0  265.0  3296.0  33.0   7.0  4.0   
Alex Smith          10.0  334.0   505.0  341.0  4042.0  26.0   5.0  0.0   
Cam Newton          11.0  331.0   492.0  291.0  3302.0  22.0  16.0  0.0   
Matthew Stafford     7.0  326.0   565.0  371.0  4446.0  29.0  10.0  1.0   
Kirk Cousins         0.0  322.0   540.0  347.0  4093.0  27.0  13.0  0.0   
Philip Rivers        9.0  320.0   575.0  360.0  4515.0  28.0  10.0  0.0   
Ben Roethlisberger   9.0  310.0   561.0  360.0  4251.0  28.0  14.0  2.0   
Jared Goff           8.0  305.0   477.0  296.0  3804.0  28.0   7.0  0.0   
Drew Brees           0.0  298.0   536.0  386.0  4334.0  23.0   8.0  0.0   
Dak Prescott         6.0  295.0   490.0  308.0  3324.0  22.0  13.0  0.0   
Blake Bortles        8.0  280.0   523.0  315.0  3687.0  21.0  13.0  3.0   
Case Keenum          9.0  274.0   481.0  325.0  3547.0  22.0   7.0  1.0   
Matt Ryan            0.0  256.0   529.0  342.0  4095.0  20.0  12.0  0.0   
Andy Dalton          6.0  252.0   496.0  297.0  3320.0  25.0  12.0  0.0   
Derek Carr          10.0  244.0   515.0  323.0  3496.0  22.0  13.0  2.0   
Tyrod Taylor         6.0  239.0   420.0  263.0  2799.0  14.0   4.0  0.0   
Jameis Winston       1.0  236.0   442.0  282.0  3504.0  19.0  11.0  1.0   
Josh McCown         11.0  235.0   397.0  267.0  2926.0  18.0   9.0  0.0   
Marcus Mariota       8.0  226.0   453.0  281.0  3232.0  13.0  15.0  0.0   
Eli Manning          8.0  223.0   571.0  352.0  3468.0  19.0  13.0  0.0   
Jacoby Brissett     11.0  220.0   469.0  276.0  3098.0  13.0   7.0  0.0   
Joe Flacco          10.0  212.0   549.0  352.0  3141.0  18.0  13.0  1.0   
Deshaun Watson       7.0  204.0   204.0  126.0  1699.0  19.0   8.0  1.0   
DeShone Kizer        9.0  188.0   476.0  255.0  2894.0  11.0  22.0  0.0   
Jay Cutler           1.0  186.0   429.0  266.0  2666.0  19.0  14.0  0.0   
Aaron Rodgers        8.0  157.0   238.0  154.0  1675.0  16.0   6.0  0.0   
Trevor Siemian       0.0  143.0   349.0  206.0  2285.0  12.0  14.0  0.0   
Mitchell Trubisky    9.0  142.0   330.0  196.0  2193.0   7.0   7.0  0.0   
...                  ...    ...     ...    ...     ...   ...   ...  ...   
T.J. Yates           7.0   41.0    97.0   47.0   523.0   4.0   3.0  0.0   
Kevin Hogan          9.0   38.0    75.0   46.0   517.0   4.0   5.0  0.0   
Sam Bradford         9.0   32.0    43.0   32.0   382.0   3.0   0.0  0.0   
Bryce Petty         11.0   24.0   112.0   55.0   544.0   1.0   3.0  0.0   
David Fales          1.0   20.0    43.0   29.0   265.0   1.0   1.0  0.0   
Paxton Lynch         0.0   18.0    45.0   30.0   295.0   2.0   3.0  0.0   
E.J. Manuel         10.0   15.0    43.0   24.0   265.0   1.0   1.0  0.0   
Geno Smith           8.0   13.0    36.0   21.0   212.0   1.0   0.0  0.0   
Ryan Mallett        10.0   13.0    16.0    9.0    56.0   2.0   0.0  0.0   
Nathan Peterman      6.0   12.0    49.0   24.0   252.0   2.0   5.0  0.0   
Landry Jones         9.0   12.0    28.0   23.0   239.0   1.0   1.0  0.0   
Patrick Mahomes     10.0   10.0    35.0   22.0   284.0   0.0   1.0  0.0   
Nate Sudfeld        10.0    7.0    23.0   19.0   134.0   0.0   0.0  0.0   
Sean Mannion         8.0    5.0    37.0   22.0   185.0   0.0   0.0  0.0   
Matt Cassel          8.0    5.0    42.0   25.0   162.0   1.0   2.0  0.0   
Joe Webb             6.0    3.0     7.0    2.0    35.0   0.0   1.0  0.0   
Cody Kessler         9.0    2.0    23.0   11.0   126.0   0.0   1.0  0.0   
Scott Tolzien       11.0    1.0    18.0    9.0   128.0   0.0   2.0  0.0   
Cooper Rush          6.0    1.0     3.0    1.0     2.0   0.0   0.0  0.0   
A.J. McCarron        6.0    1.0    14.0    7.0    66.0   0.0   0.0  0.0   
Chad Henne           8.0    0.0     2.0    0.0     0.0   